In [1]:
# # Monter le stockage dans Colab
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# Imports
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm

In [2]:
# Set batch size for training and validation
batch_size = 32
num_classes = 3
num_iterations = 34

In [3]:
# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [4]:
# # Chemin d'accès dans Colab
# train = '/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/data/Cropped/RGB/train'
# test = '/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/data/Cropped/RGB/test'
# validation = '/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/data/Cropped/RGB/val'

In [5]:
# Chemin d'accès en local
train = "./data/Cropped/RGB/train"
test = "./data/Cropped/RGB/test"
validation = "./data/Cropped/RGB/val"

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(train, seed=5, batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(test, seed=5, batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(validation, seed=5, batch_size=batch_size)

Found 1086 files belonging to 3 classes.
Found 60 files belonging to 3 classes.
Found 60 files belonging to 3 classes.


In [7]:
# Afficher des les classes et leurs labels
import os
categories=os.listdir(train)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

img_size=100
data=[]
target=[]

{'Semi': 0, 'Open': 1, 'Closed': 2}
['Semi', 'Open', 'Closed']
[0, 1, 2]


In [29]:
import cv2
import matplotlib.pyplot as plt

for i in train:
    im = cv2.imread(i)
    im_resized = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)

    plt.imshow(cv2.cvtColor(im_resized, cv2.COLOR_BGR2RGB))
    plt.show()

TypeError: Can't convert object of type 'tuple' to 'str' for 'filename'

In [8]:
def normalize_img(image, label, img_size):
    # Resize image to the desired img_size and normalize it
    # One hot encode the label
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.
    label = tf.one_hot(label, depth=num_classes)
    return image, label
    
def preprocess_data(train, validation, batch_size, img_size):
    # Apply the normalize_img function on all train and validation data and create batches
    train_processed = train.map(lambda image, label: normalize_img(image, label, img_size))
    
    # If your data is already batched (eg, when using the image_dataset_from_directory function), remove .batch(batch_size)
    train_processed = train_processed.repeat()
    
    validation_processed = validation.map(lambda image, label: normalize_img(image, label, img_size))
    
    # If your data is already batched (eg, when using the image_dataset_from_directory function), remove .batch(batch_size)
    validation_processed = validation_processed
    
    return train_processed, validation_processed
    
# Run preprocessing
train_processed_224, validation_processed_224 = preprocess_data(train_ds, val_ds, batch_size, img_size=[224,224])
train_processed_331, validation_processed_331 = preprocess_data(train_ds, val_ds, batch_size, img_size=[331,331])


In [10]:
# Loop over each model available in Keras
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}

for model_name, model in tqdm(model_dictionary.items()):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)
    if 'NASNetLarge' in model_name:
        input_shape=(331,331,3)
        train_processed = train_processed_331
        validation_processed = validation_processed_331
    else:
        input_shape=(224,224,3)
        train_processed = train_processed_224
        validation_processed = validation_processed_224
        
    # load the pre-trained model with global average pooling as the last layer and freeze the model weights
    pre_trained_model = model(include_top=False, pooling='avg', input_shape=input_shape)
    pre_trained_model.trainable = False
    
    # custom modifications on top of pre-trained model and fit
    clf_model = tf.keras.models.Sequential()
    clf_model.add(pre_trained_model)
    clf_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    clf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    history = clf_model.fit(train_processed, epochs=10, validation_data=validation_processed, steps_per_epoch=num_iterations)
    
    # Calculate all relevant metrics
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
    model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])

  0%|                                                                                 | 0/35 [00:00<?, ?it/s]

Epoch 1/10
34/34 [==============================] - 135s 4s/step - loss: 0.6847 - accuracy: 0.7422 - val_loss: 0.9703 - val_accuracy: 0.6167
Epoch 2/10
34/34 [==============================] - 139s 4s/step - loss: 0.4879 - accuracy: 0.8287 - val_loss: 0.6989 - val_accuracy: 0.6667
Epoch 3/10
34/34 [==============================] - 123s 4s/step - loss: 0.3982 - accuracy: 0.8554 - val_loss: 0.6822 - val_accuracy: 0.6833
Epoch 4/10
34/34 [==============================] - 141s 4s/step - loss: 0.3460 - accuracy: 0.8858 - val_loss: 0.6287 - val_accuracy: 0.6833
Epoch 5/10
34/34 [==============================] - 132s 4s/step - loss: 0.3054 - accuracy: 0.8987 - val_loss: 0.4879 - val_accuracy: 0.7667
Epoch 6/10
34/34 [==============================] - 130s 4s/step - loss: 0.2778 - accuracy: 0.9061 - val_loss: 0.5634 - val_accuracy: 0.7333
Epoch 7/10
19/34 [===============>..............] - ETA: 53s - loss: 0.2590 - accuracy: 0.9227

  0%|                                                                                 | 0/35 [14:39<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Convert Results to DataFrame for easy viewing
benchmark_df = pd.DataFrame(model_benchmarks)

# sort in ascending order of num_model_params column
benchmark_df.sort_values('num_model_params', inplace=True)

# write results to csv file
benchmark_df.to_csv('benchmark_df.csv', index=False)
# benchmark_df

# # benchmark_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/benchmark_df.csv')
# benchmark_df = pd.read_csv('benchmark_df.csv')
# benchmark_df.sort_values(by=['validation_accuracy'], inplace=True, ascending=False)
# benchmark_df

In [ ]:
# Sauvegarder le dataframe du Benchmark des models
benchmark_df.to_csv('benchmark_df.csv', index=False)
benchmark_df.sort_values(by=['validation_accuracy'], inplace=True, ascending=False)
benchmark_df

In [ ]:
# # Sauvegarder le dataframe du Benchmark des models
# from google.colab import files
# files.download("benchmark_df.csv", index=False)

In [ ]:
# Loop over each row and plot the num_model_params vs validation_accuracy
markers=[".",",","o","v","^","<",">","1","2","3","4","8","s","p","P","*","h","H","+","x","X","D","d","|","_",4,5,6,7,8,9,10,11]
plt.figure(figsize=(7,5))

for row in benchmark_df[0:5].itertuples():
    plt.scatter(row.num_model_params, row.validation_accuracy, label=row.model_name, marker=markers[row.Index], s=150, linewidths=0)
    
plt.xscale('log')
plt.xlabel('Number of Parameters in Model')
plt.ylabel('Validation Accuracy after 3 Epochs')
plt.title('Accuracy vs Model Size')

# Move legend out of the plot
plt.legend(bbox_to_anchor=(1, 1), loc='upper left');

In [ ]:
# Loop over each row and plot the num_model_params vs validation_accuracy

plt.figure(figsize=(10,8))

for row in benchmark_df.itertuples():
    plt.scatter(row.num_model_params, row.validation_accuracy, label=row.model_name, s=150)
    
plt.xscale('log')
plt.xlabel('Number of Parameters in Model')
plt.ylabel('Validation Accuracy after 10 Epochs')
plt.title('Accuracy vs Model Size')

# Move legend out of the plot
plt.legend(bbox_to_anchor=(1, 1), loc='upper left');

In [ ]:
# importation de MobileNet

mobilenet = tf.keras.applications.mobilenet.MobileNet(
    input_shape=None, alpha=1.0, depth_multiplier=1, dropout=0.001,
    include_top=True, weights='imagenet', input_tensor=None, pooling=None,
    classes=1000, classifier_activation='softmax'
)

In [ ]:
# Compile model
mobilenet.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
          metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224,224,3), include_top=True, weights='imagenet', pooling="avg",
    classifier_activation='softmax'
    )

In [ ]:
plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.savefig('/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/captures/history.png')
plt.savefig('/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/captures/history.pdf')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('accuracy')
plt.legend()
plt.savefig('/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/captures/accuracy.png')
plt.savefig('/content/drive/MyDrive/Colab Notebooks/00.Cas_pratique_ia/captures/accuracy.pdf')
plt.show()

In [ ]:
print(model.evaluate(test_data,test_target))

In [ ]:
model.summary()